Input track is a centerline csv file in the same folder as this, returns npy files for centerline and racing line paths

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import scipy
from scipy.interpolate import CubicSpline
from scipy.interpolate import PchipInterpolator
from shapely.geometry import Point, Polygon
from scipy.signal import savgol_filter
from scipy.interpolate import interp1d

In [ ]:
def calculate_path(track):
    data = pd.read_csv(track)
    data.head()
    # Extract x and y columns
    x_raw = data["# x_m"].values
    y_raw = data[" y_m"].values

    # Ensure equal spacing along centerline
    distances = np.cumsum(np.sqrt(np.diff(x_raw, prepend=x_raw[0])**2 + np.diff(y_raw, prepend=y_raw[0])**2))
    uniform_distances = np.linspace(0, distances[-1], len(x_raw))

    # Interpolate smoothed centerline
    cs_x = CubicSpline(distances, x_raw)
    cs_y = CubicSpline(distances, y_raw)

    x = cs_x(uniform_distances)
    y = cs_y(uniform_distances)

    # Compute tangent vectors (dx, dy)
    dx = np.gradient(x)
    dy = np.gradient(y)
    lengths = np.sqrt(dx**2 + dy**2)

    # Normalize to get unit tangent vectors
    dx /= lengths
    dy /= lengths

    # Compute normal vectors (-dy, dx) (perpendicular to tangents)
    nx = -dy
    ny = dx

    # Scale normal vectors to half the track width (1.1m)
    offset = 1.1
    offset2 = 0.6
    left_x = x + offset * nx
    left_y = y + offset * ny
    right_x = x - offset * nx
    right_y = y - offset * ny

    left_x2 = x + offset2 * nx
    left_y2 = y + offset2 * ny
    right_x2 = x - offset2 * nx
    right_y2 = y - offset2 * ny


    ddx = np.gradient(dx)
    ddy = np.gradient(dy)

    # Compute curvature (κ = (dx * ddy - dy * ddx) / (dx^2 + dy^2)^(3/2))
    curvature = np.abs(dx * ddy - dy * ddx) / (dx**2 + dy**2)**(3/2)

    corner_threshold = 0.06  # Slightly stricter for corners
    

    # Identify corners and hairpins again
    is_corner = curvature > corner_threshold
   

    # Identify start and end of corners
    corner_start_indices = np.where(np.diff(is_corner.astype(int)) == 1)[0] + 1
    corner_end_indices = np.where(np.diff(is_corner.astype(int)) == -1)[0] + 1

    # Handle edge case where track starts or ends in a corner
    if is_corner[0]:  # If first point is in a corner
        corner_start_indices = np.insert(corner_start_indices, 0, 0)
    if is_corner[-1]:  # If last point is in a corner
        corner_end_indices = np.append(corner_end_indices, len(curvature) - 1)

    # Pair up start and end indices as (start, end) for each corner
    corner_segments = list(zip(corner_start_indices, corner_end_indices))




    # Compute heading angles
    dx = np.diff(x)
    dy = np.diff(y)
    # Compute heading angles
    theta = np.arctan2(np.diff(y), np.diff(x))

    # Compute angle difference (ensure continuity)
    d_theta = np.diff(theta)
    d_theta = np.unwrap(d_theta)  # Fixes jumps at -π/π boundary

    # Classify corners
    corner_directions = []
    for start, end in corner_segments:
        avg_turn = np.mean(d_theta[start:end-1])  # Compute average direction change
        if avg_turn > 0:
            corner_directions.append("Left Turn")
        else:
            corner_directions.append("Right Turn")

    # Plot the track with categorized sections






    corner_type = [None] * len(corner_segments)

    for i, (start, end) in enumerate(corner_segments):
        if i + 1 >= len(corner_segments):
            n = 0
        else: 
            n = i + 1
        start_n, end_n = corner_segments[n]
        distance = np.sqrt((x[end] - x[start_n]) ** 2 + (y[end] - y[start_n]) ** 2)

        if distance > 10:
            corner_type[i] = "Early Apex"
        elif corner_directions[i] != corner_directions[n]:
            corner_type[i] = "Late Apex"
        elif corner_directions[i] == corner_directions[n]:
            corner_type[i] = "Geometric Apex"
        
        # New hairpin definition
        heading_start = theta[start]
        heading_end = theta[end - 1]  # theta is len(x)-1
        total_heading_change = np.abs(np.arctan2(np.sin(heading_end - heading_start), np.cos(heading_end - heading_start)))
        corner_length = np.sqrt((x[end] - x[start])**2 + (y[end] - y[start])**2)

        # Define thresholds
        sharp_heading_threshold = np.deg2rad(90)
        hairpin_heading_threshold = np.deg2rad(150)
        length_threshold = 6  # in meters, adjust as needed

        if total_heading_change > hairpin_heading_threshold:
            corner_type[i] = "Hairpin"
        elif total_heading_change > sharp_heading_threshold and corner_length < length_threshold:
            corner_type[i] = "Sharp Turn"
        
    # Define unique colors for corner types
    corner_colors = {
        "Early Apex": "red",
        
        "Geometric Apex": "blue",
        
        "Late Apex": "yellow",
        
        "Hairpin": "purple",

        "Sharp Turn": "orange"
        
    }

    # Keep track of added labels to prevent duplicates
    added_labels = set()





    plt.show()
    # Precompute tangents and normals
    dx = np.gradient(x)
    dy = np.gradient(y)
    lengths = np.sqrt(dx**2 + dy**2)
    dx /= lengths
    dy /= lengths
    nx = -dy
    ny = dx

    brake_list = []
    apex_list = []
    exit_list = []
    collect = []

    for i, (start, end) in enumerate(corner_segments):
        # Direction offset
        dir_offset = -0.7 if corner_directions[i] == "Left Turn" else 0.7
        dir_offset2 = -0.1 if corner_directions[i] == "Left Turn" else 0.1
        exit_dist = 3
        exit_offset = 0
        # Corner type settings
        if corner_type[i] == "Late Apex":
            brake_dist, apex_pos, exit_dist = -1, 3, -1
            exit_offset = 0
        elif corner_type[i] == "Early Apex":
            brake_dist, apex_pos = 4, -2
            exit_offset = -1
            dir_offset2 = -0.1 if corner_directions[i] == "Left Turn" else -0.1
            exit_dist = 0
        elif corner_type[i] == "Geometric Apex":
            brake_dist, apex_pos = 3, 0
        elif corner_type[i] == "Hairpin":
            brake_dist, apex_pos = -4, 0
            dir_offset2, exit_dist = 0, -5
            exit_offset = 1
        elif corner_type[i] == "Sharp Turn":
            brake_dist, apex_pos = 1, -1
            dir_offset2 = 0
            exit_dist = -2
            exit_offset = 1
        # Use previous corner to assess spacing
        prev_i = i - 1 if i - 1 >= 0 else 0
        start_n, end_n = corner_segments[prev_i]
        distance = np.linalg.norm([x[end_n] - x[start], y[end_n] - y[start]])

        # Safe indexing
        s_brake = max(start - brake_dist, 0)
        e_exit = min(end + exit_dist, len(x) - 1)
        apex_idx = int((start + end) / 2) + apex_pos
        apex_idx = np.clip(apex_idx, 0, len(x) - 1)

        # Append points
        brake = (x[s_brake] + dir_offset * nx[s_brake], y[s_brake] + dir_offset * ny[s_brake])
        apex = (x[apex_idx] - dir_offset2 * nx[apex_idx], y[apex_idx] - dir_offset2 * ny[apex_idx])
        exit_ = (x[e_exit], y[e_exit])
        if exit_offset != 0:
            exit_ = (x[e_exit] + dir_offset * nx[e_exit], y[e_exit] + dir_offset * ny[e_exit])
        
        
        #if distance < 2:
        #    brake = (x[start] + dir_offset * nx[start], y[start] + dir_offset * ny[start])

        #brake_list.append(brake)
        apex_list.append(apex)
        exit_list.append(exit_)
        if distance >2:
            brake_list.append(brake)
        collect.extend([brake, apex, exit_])

        # If long straight ahead, add push-out point
        next_i = i + 1 if i + 1 < len(corner_segments) else 0
        start_next, _ = corner_segments[next_i]
        distance_next = np.linalg.norm([x[end] - x[start_next], y[end] - y[start_next]])
        if distance_next > 30:
            e_straight = min(end + 40, len(x) - 1)
            pushout = (x[e_straight] + dir_offset * nx[e_straight], y[e_straight] + dir_offset * ny[e_straight])
            collect.append(pushout)

    # Convert to arrays
    brake_array = np.array(brake_list)
    apex_array = np.array(apex_list)
    exit_array = np.array(exit_list)

    # Close the loop

    #collect.append((0,0))
    #collect.insert(0,[0,0])
    #collect.pop()
    collect.append((0,0))
    racing_line_points = np.array(collect)
 
    
    def compute_arc_length(points):
        dists = np.sqrt(np.sum(np.diff(points, axis=0)**2, axis=1))
        arc_length = np.insert(np.cumsum(dists), 0, 0)
        return arc_length
    def densify_near_apexes(points, apexes, radius=3, num_extra=2):
        new_points = [points[0]]
        for i in range(1, len(points)):
            p0 = np.array(points[i - 1])
            p1 = np.array(points[i])
            segment = [p0 + t * (p1 - p0) for t in np.linspace(0, 1, 2)]  # default

            # Check if this segment is close to any apex
            mid = (p0 + p1) / 2
            if any(np.linalg.norm(mid - np.array(ap)) < radius for ap in apexes):
                # Insert more points in this segment
                segment = [p0 + t * (p1 - p0) for t in np.linspace(0, 1, num_extra)]

            new_points.extend(segment[1:])  # Avoid repeating the previous point
        return np.array(new_points)

    racing_line_points_dense = densify_near_apexes(racing_line_points, apex_list)
    arc_len = compute_arc_length(racing_line_points_dense)


    # Create a parameterization (like cumulative distance)
    distances = np.cumsum(np.sqrt(np.sum(np.diff(racing_line_points_dense, axis=0)**2, axis=1)))
    distances = np.insert(distances, 0, 0)

    # Step 4: PCHIP interpolation
    
    pchip_x = PchipInterpolator(arc_len, racing_line_points_dense[:, 0])
    pchip_y = PchipInterpolator(arc_len, racing_line_points_dense[:, 1])

    arc_len_fine = np.linspace(arc_len[0], arc_len[-1], 2000)
    x_racing = pchip_x(arc_len_fine)
    y_racing = pchip_y(arc_len_fine)
    #x_close, y_close = splev(u_fine, tck)
    x_close = np.linspace(collect[-1][0], collect[0][0], 10)
    y_close = np.linspace(collect[-1][1], collect[0][1], 10)
    # Append the closed loop points to the racing line
    x_racing = np.concatenate((x_racing, x_close))
    y_racing = np.concatenate((y_racing, y_close))

    # Construct and close track boundary polygon
    
    boundary_coords = np.vstack([
        np.column_stack((left_x2, left_y2)),
        np.column_stack((right_x2[::-1], right_y2[::-1]))
    ])
    if not np.allclose(boundary_coords[0], boundary_coords[-1]):
        boundary_coords = np.vstack([boundary_coords, boundary_coords[0]])
    track_poly = Polygon(boundary_coords)

    # Filter spline to keep only valid (inside) points
    valid_points = [(xr, yr) for xr, yr in zip(x_racing, y_racing) if track_poly.contains(Point(xr, yr))]
    invalid_points = [(xr, yr) for xr, yr in zip(x_racing, y_racing) if not track_poly.contains(Point(xr, yr))]

    x_racing_valid, y_racing_valid = zip(*valid_points)


    def project_to_track(xr, yr, centerline_x, centerline_y, track_poly):
        point = np.array([xr, yr])
        # Find the closest centerline point
        dists = np.sqrt((centerline_x - xr)**2 + (centerline_y - yr)**2)
        idx = np.argmin(dists)
        
        # Compute normal vector at closest point
        dx = np.gradient(centerline_x)
        dy = np.gradient(centerline_y)
        lengths = np.sqrt(dx**2 + dy**2)
        dx /= lengths
        dy /= lengths
        nx = -dy
        ny = dx

        # Push point back toward centerline along negative normal
        for scale in np.linspace(0, 1.5, 30):  # Gradually push it back
            new_point = point * (1 - scale) + scale * np.array([centerline_x[idx], centerline_y[idx]])
            if track_poly.contains(Point(*new_point)):
                return new_point[0], new_point[1]
        
        # As fallback, just return the centerline point
        return centerline_x[idx], centerline_y[idx]

    # Fix invalid points
    fixed_points = []
    for xr, yr in zip(x_racing, y_racing):
        if track_poly.contains(Point(xr, yr)):
            fixed_points.append((xr, yr))
        else:
            xr_fixed, yr_fixed = project_to_track(xr, yr, x, y, track_poly)
            fixed_points.append((xr_fixed, yr_fixed))

    x_racing_fixed, y_racing_fixed = zip(*fixed_points)
    
    # Smooth x and y coordinates independently
    x_smooth = savgol_filter(x_racing_fixed, window_length=11, polyorder=3)
    y_smooth = savgol_filter(y_racing_fixed, window_length=11, polyorder=3)

    # Compute cumulative distances along the path
    spline = np.column_stack((x_smooth, y_smooth))
    deltas = np.diff(spline, axis=0)
    segment_lengths = np.linalg.norm(deltas, axis=1)
    arc_lengths = np.insert(np.cumsum(segment_lengths), 0, 0.0)
    interp_x = interp1d(arc_lengths, spline[:, 0], kind='cubic')
    interp_y = interp1d(arc_lengths, spline[:, 1], kind='cubic')
    desired_spacing = 0.1  # meters between points (tune this)
    new_arc_lengths = np.arange(0, arc_lengths[-1], desired_spacing)
    resampled_x = interp_x(new_arc_lengths)
    resampled_y = interp_y(new_arc_lengths)
    resampled_racing = np.column_stack((resampled_x, resampled_y))
    # Plotting
    plt.figure(figsize=(10, 6))
    plt.scatter(brake_array[:, 0], brake_array[:, 1], color="red", label="Brake", s=50, marker="o")
    plt.scatter(apex_array[:, 0], apex_array[:, 1], color="blue", label="Apex", s=50, marker="x")
    plt.scatter(exit_array[:, 0], exit_array[:, 1], color="green", label="Exit", s=50, marker="s")
    plt.plot(resampled_x, resampled_y, label="Racing Line", color="red", linestyle='-')
    plt.plot(left_x, left_y, label="Left Boundary", color="black", linestyle="-")
    plt.plot(right_x, right_y, label="Right Boundary", color="black", linestyle="-")
    for i, (start, end) in enumerate(corner_segments):
        label = f"{corner_type[i]}"
        color = corner_colors.get(label, "gray")  # Default to gray if not listed
        
        # Only add legend label once
        if label not in added_labels:
            plt.scatter(x[start:end], y[start:end], label=label, color=color, linewidths=5, alpha=0.5, capstyle='butt')
            added_labels.add(label)
        else:
            plt.scatter(x[start:end], y[start:end], color=color, linewidths=5, alpha=0.5, capstyle='butt')

    
    plt.title("Optimized Racing Line")
    plt.legend()
    plt.axis("equal")
    plt.grid(True)
    plt.show()

    # Resample the  center line to ensure uniform spacing
    spline = np.column_stack((x, y))
    deltas = np.diff(spline, axis=0)
    segment_lengths = np.linalg.norm(deltas, axis=1)
    arc_lengths = np.insert(np.cumsum(segment_lengths), 0, 0.0)
    interp_x = interp1d(arc_lengths, spline[:, 0], kind='cubic')
    interp_y = interp1d(arc_lengths, spline[:, 1], kind='cubic')
    desired_spacing = 0.1  # meters between points (tune this)
    new_arc_lengths = np.arange(0, arc_lengths[-1], desired_spacing)
    resampled_x = interp_x(new_arc_lengths)
    resampled_y = interp_y(new_arc_lengths)

    resampled_center = np.column_stack((resampled_x, resampled_y))
    np.save('racing_line.npy', resampled_racing)
    np.save('center_line.npy', resampled_center)
    return resampled_racing, resampled_center